In [28]:
import numpy as np

np.random.seed(42)
data = np.random.randint(-100, 100, size=10)

def modified_relu(data, embed_size=5)

    result = np.empty(10)

    alice = np.random.randint(-100, 100, size=10)
    bob = data - alice

    alice_mask = alice >= 0
    bob_mask = bob >= 0

    false_indices = np.where(alice_mask != bob_mask)[0]
    true_indices = np.where(alice_mask == bob_mask)[0]
    same_result = alice_mask[true_indices] + bob_mask[true_indices]

    keys = np.random.randint(0, embed_size, size=len(false_indices))

    # print(keys)
    embedded_alice = np.random.randint(-100, 100, size=(embed_size, len(false_indices)))
    embedded_alice[keys, range(len(false_indices))] = alice[false_indices]

    embedded_bob = embedded_alice + bob[false_indices]

    mask = embedded_bob > 0
    dif_result = mask[keys, range(len(false_indices))]
    result[true_indices] = same_result
    result[false_indices] = dif_result

    print(f"Accuracy:{((data > 0) == result).sum() * 100 / len(data) : .2f}%")
    
    return result

Accuracy: 100.00%
